In [1]:
import yfinance as yf
import matplotlib.pyplot as plt
import plotly.express as px

# Fetch stock data
ticker = "NVDA"
stock = yf.download(ticker, period="6mo")

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


In [22]:
df = stock.copy()

In [23]:
df.columns = df.columns.droplevel(1)

In [24]:
df

Price,Close,High,Low,Open,Volume
Date,,,,,
2024-09-16,116.761208,118.160984,114.341599,116.771208,248772300
2024-09-17,115.571404,118.780894,114.811531,118.150990,231925900
2024-09-18,113.351761,117.681058,113.201783,115.871352,310318900
2024-09-19,117.851044,119.640757,117.231141,117.331123,293506400
2024-09-20,115.981339,118.600920,115.371436,117.041166,382462400
...,...,...,...,...,...
2025-03-10,106.970161,111.839709,105.450297,109.889891,366487400
2025-03-11,108.750000,112.229676,104.760361,106.980159,354865700
2025-03-12,115.739998,116.760002,112.879997,114.120003,323857500


In [3]:
stock.index

DatetimeIndex(['2024-09-16', '2024-09-17', '2024-09-18', '2024-09-19',
               '2024-09-20', '2024-09-23', '2024-09-24', '2024-09-25',
               '2024-09-26', '2024-09-27',
               ...
               '2025-03-03', '2025-03-04', '2025-03-05', '2025-03-06',
               '2025-03-07', '2025-03-10', '2025-03-11', '2025-03-12',
               '2025-03-13', '2025-03-14'],
              dtype='datetime64[ns]', name='Date', length=124, freq=None)

In [7]:
stock[('Close', 'NVDA')]

Date
2024-09-16    116.761208
2024-09-17    115.571404
2024-09-18    113.351761
2024-09-19    117.851044
2024-09-20    115.981339
                 ...    
2025-03-10    106.970161
2025-03-11    108.750000
2025-03-12    115.739998
2025-03-13    115.580002
2025-03-14    121.669998
Name: (Close, NVDA), Length: 124, dtype: float64

In [25]:
 # Plot stock price
fig = px.line(df, 
              x=df.index, 
              y=['Close'], #, "SMA_50", "SMA_200"], 
              title=f"{ticker} Stock Price & SMAs")

fig.show()

In [10]:
import plotly

plotly.__version__

'6.0.0'